In [ ]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2505177 sha256=3243894f8b2fdc6c02b2a4f03f4f993ff32287728aa6877ce265ef287d8bb06c
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [ ]:
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
import ipywidgets as widgets
from IPython.display import display

# Load the MovieLens 100K dataset
file_path = 'u.data'  # Change this path to your dataset location

# Load data into a DataFrame
columns = ['userId', 'movieId', 'rating', 'timestamp']
df = pd.read_csv(file_path, sep='\t', names=columns)

# Display the first few rows of the dataset
print("MovieLens 100K Dataset:")
print(df.head())

# Prepare data for Surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)

# Split the dataset into training and test sets
trainset, testset = train_test_split(data, test_size=0.2)

# Use SVD for matrix factorization
algo = SVD()
algo.fit(trainset)

# Function to get movie recommendations for a specific user
def get_recommendations(user_id, n_recommendations=10):
    # Get a list of all movie IDs
    all_movie_ids = df['movieId'].unique()

    # Get the movies already rated by the user
    rated_movies = df[df['userId'] == user_id]['movieId'].tolist()

    # Predict ratings for all movies not yet rated by the user
    predictions = []
    for movie_id in all_movie_ids:
        if movie_id not in rated_movies:
            pred = algo.predict(user_id, movie_id)
            predictions.append((movie_id, pred.est))

    # Sort predictions by estimated rating and return top N movies
    recommendations = sorted(predictions, key=lambda x: x[1], reverse=True)[:n_recommendations]
    return [movie_id for movie_id, _ in recommendations]

# Create a dropdown widget for user selection
user_id_widget = widgets.Dropdown(
    options=df['userId'].unique(),
    description='User ID:',
    value=1  # Default value
)

# Function to display recommendations when user selects an ID
def on_user_id_change(change):
    user_id = change['new']
    recommendations = get_recommendations(user_id)
    print(f"\nRecommended movies for User {user_id}: {recommendations}")

# Link the dropdown to the function
user_id_widget.observe(on_user_id_change, names='value')

# Display the dropdown widget
display(user_id_widget)


MovieLens 100K Dataset:
   userId  movieId  rating  timestamp
0     196      242       3  881250949
1     186      302       3  891717742
2      22      377       1  878887116
3     244       51       2  880606923
4     166      346       1  886397596


Dropdown(description='User ID:', index=118, options=(196, 186, 22, 244, 166, 298, 115, 253, 305, 6, 62, 286, 2…


Recommended movies for User 305: [657, 515, 516, 276, 8, 137, 525, 185, 498, 641]
